# 주제 : OP.GG 사용자 페이지 전적 크롤링
# 대상 사이트 : op.gg
#### 데이터 수집 과정 : 
1. 게임 플레이어의 이름을 입력받아 op.gg라는 사이트의 플레이어 화면으로 접속
2. 플레이어의 게임 전적 갱신버튼 클릭
3. 갱신한지 2분 이내에는 재갱신이 불가능해 alert창이 뜨는걸 닫아줌
3. 검색결과에서 크롤링
    * 경우 1: 입력된 플레이어의 킬, 데스, 어시스트, 승/패 여부, 사용 챔피언의 정보를 가져오고 bs4를 이용해 종류별로 구분.
    * 경우 2: 입력된 플레이어의 현재 솔로랭크, 자유랭크 티어 정보를 가져옴
4. tabulate를 이용해 DataFrame을 예쁘게 정리하여 출력

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.alert import Alert
import requests
import numpy as np
import pandas as pd
from tabulate import tabulate

In [3]:
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
# options.add_argument("headless")
driver = webdriver.Chrome("/Users/goyoonjae/Desktop/dongmonitor/chromedriver",options=options)

## 입력한 사용자의 최근 20게임의 전적(승/패, 킬, 데스, 어시스트, 챔피언) 크롤링

In [4]:
def gamelog(nickname):

    driver = webdriver.Chrome("/Users/goyoonjae/Desktop/dongmonitor/chromedriver",options=options)
    url = 'https://www.op.gg/summoner/userName='
    # inp = input('롤 닉넴 입력 : ')
    url = url+nickname
    driver.get(url)
    alert = Alert(driver)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html,'html.parser')
    pd.set_option('expand_frame_repr',False)

    driver.find_element_by_css_selector('#SummonerRefreshButton').click()

    try:
        # time.sleep(3)
        alert = Alert(driver)
        # alert = driver.switch_to.alert
        
        print(alert)
        print('alert detected!')
        alert.accept()
        alert.accept()
    except:
        "noal"
    kda = soup.select('.GameItem>.Content>.KDA>.KDA>span')
    kdas = []
    for tag in kda:
        kdas.append(tag.get_text().strip())
    KDA = np.reshape(kdas,(20,3))
    print('kda 조회중...')
    # print(KDA)


    gameresult = soup.select('.GameStats>.GameResult')
    gameresults = []
    for tag in gameresult:
        gameresults.append(tag.get_text().strip())
    GRES = np.reshape(gameresults,(20,1))
    print('게임결과 조회중...')
    # print(GRES)


    champion = soup.select('.Content>.GameSettingInfo>.ChampionName>a')
    champions = []
    for tag in champion:
        champions.append(tag.get_text())
    CHAMP = np.reshape(champions,(20,1))
    print('챔피언 조회중...')


    data = pd.DataFrame(CHAMP)
    subdf = pd.DataFrame(GRES)
    data = pd.concat([data, subdf],axis = 1)
    subdf = pd.DataFrame(KDA)
    data = pd.concat([data,subdf],axis = 1)
    data.columns = ['CHAMP','RESULT','KILL','DEATH','ASSIST']
    data = tabulate(data,headers='keys',tablefmt='pretty',showindex=range(1,21))
    

    driver.delete_all_cookies()
    driver.quit()
    print('정보반환완료')
    print(data)
#     return data
    

## 입력한 사용자의 현재 솔로랭크, 자유랭크 티어 크롤링

In [5]:
def nowtier(nickname):
    driver = webdriver.Chrome("/Users/goyoonjae/Desktop/dongmonitor/chromedriver",options=options)
    url = 'https://www.op.gg/summoner/userName='
    # inp = input('롤 닉넴 입력 : ')
    url = url+nickname
    driver.get(url)
    alert = Alert(driver)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html,'html.parser')

    driver.find_element_by_css_selector('#SummonerRefreshButton').click()

    try:
        # time.sleep(3)
        alert = Alert(driver)
        # alert = driver.switch_to.alert
        
        print(alert)
        # time.sleep(1)
        print('alert detected!')
        alert.accept()
        # time.sleep(1)
        alert.accept()
    except:
        "noal"
        # time.sleep(3)
        # print('no alert!')

    
    soltier = soup.find(class_= 'TierRank').get_text()
    subtier = soup.find(class_="sub-tier__rank-tier").get_text().strip()
    driver.delete_all_cookies()
    driver.quit()
    print(nickname + f'님의 현재 티어\n솔로랭크:{soltier}\n자유랭크:{subtier}')
#     return soltier, subtier


# 전적을 크롤링하고 싶은 경우

In [6]:
nickname = input('사용자의 이름을 입력해주세요')
gamelog(nickname)

사용자의 이름을 입력해주세요 함경도물냉면


alert detected!
kda 조회중...
게임결과 조회중...
챔피언 조회중...
정보반환완료
+----+----------+---------+------+-------+--------+
|    |  CHAMP   | RESULT  | KILL | DEATH | ASSIST |
+----+----------+---------+------+-------+--------+
| 1  |  Akali   | Defeat  |  4   |   6   |   3    |
| 2  | LeBlanc  | Defeat  |  5   |   5   |   5    |
| 3  |  Yasuo   | Defeat  |  4   |   8   |   3    |
| 4  |  Yasuo   | Defeat  |  0   |   5   |   0    |
| 5  | LeBlanc  | Victory |  9   |   4   |   8    |
| 6  |  Sylas   | Defeat  |  7   |   9   |   9    |
| 7  |  Yasuo   | Victory |  9   |   5   |   7    |
| 8  |   Yone   | Defeat  |  4   |   8   |   1    |
| 9  |  Sylas   | Defeat  |  8   |   6   |   2    |
| 10 |  Yasuo   | Victory |  5   |   2   |   4    |
| 11 | Malphite | Victory |  4   |   1   |   3    |
| 12 |  Sylas   | Defeat  |  3   |   5   |   7    |
| 13 |  Garen   | Victory |  6   |   0   |   0    |
| 14 |   Sett   | Victory |  0   |   9   |   2    |
| 15 | LeBlanc  | Victory |  5   |   5   |   11   |
| 16 | 

# 티어를 크롤링하고 싶은 경우

In [7]:
nickname = input('사용자의 이름을 입력해주세요')
nowtier(nickname)

사용자의 이름을 입력해주세요 함경도물냉면


alert detected!
함경도물냉면님의 현재 티어
솔로랭크:Silver 4
자유랭크:Bronze 3
